In [8]:
import sys
import os
# Thêm thư mục gốc của project vào sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [9]:
import json
import os
import sys
# Thêm thư mục gốc của project vào sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from utils.api_client import RAGAPIClient
# Initialize API client
api_client = RAGAPIClient(base_url="http://localhost:8000/api", endpoint='query')

# Check if API is available
if not api_client.check_health():
    print("⚠️ RAG API is not available at http://localhost:8000")
    print("Please start the RAG service first!")
else:
    print("✅ RAG API is available")
def load_questions(file_path):
    """Load questions from a JSON file"""
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)




2025-12-02 11:34:55.750 | INFO     | utils.api_client:__init__:24 - Initialized RAG API Client: http://localhost:8000/api


✅ RAG API is available


In [12]:
def generate_rag_answers(questions, api_client):
    """Generate RAG answers for a list of questions"""
    rag_responses = []
    
    for i, tc in enumerate(questions):
        question = tc.get('question', '')
        print(f"[{i}] Generating answer for: {question[:50]}...")
        
        try:
            response = api_client.ask_question(question)
            contexts = [f'{source.get('text_preview')}'for source in response.get('sources', [])]
            
            rag_responses.append({
                'question_id': tc.get('id', i),
                'ground_truth': tc.get('ground_truth', ''),
                'question': question,
                'answer': response.get('answer', ''),
                'contexts': contexts,
                'processing_time': response.get('metadata', {}).get('total_time_ms', None)
            })
            
            print(f"  ✅ Got answer ({len(response.get('answer', ''))} chars)")
        
        except Exception as e:
            print(f"  ❌ Error: {e}")
            rag_responses.append({
                'question_id': tc.get('id', i),
                'question': question,
                'answer': '',
                'contexts': [],
                'error': str(e)
            })
    
    print(f"\n✅ Generated {len([r for r in rag_responses if r.get('answer')])} answers")
    return rag_responses

def save_results(rag_responses, output_path):
    """Save RAG responses to a JSON file"""
    with open(output_path, "w", encoding="utf-8-sig") as f:
        json.dump(rag_responses, f, ensure_ascii=False, indent=2)
    print(f"✅ Saved {len(rag_responses)} responses to {output_path}")

def process_folder(input_folder,result_file_name, api_client):
    """Process all JSON files in the input folder"""
    input_path = os.path.join(input_folder, "questions.json")
    output_path = os.path.join(input_folder, result_file_name)
    print(f"\n📂 Processing file: {input_path}")
    questions = load_questions(input_path)
    responses = generate_rag_answers(questions, api_client)
    save_results(responses, output_path)



In [15]:
process_folder("../data/testcase/multi_hop","results_edited_prompt.json", api_client)

2025-12-02 11:59:46.648 | INFO     | utils.api_client:ask_question:43 - Asking question: Vì sao nhóm Toán – Tin học – Khoa học tự nhiên của...



📂 Processing file: ../data/testcase/multi_hop/questions.json
[0] Generating answer for: Vì sao nhóm Toán – Tin học – Khoa học tự nhiên của...
{'query': 'Vì sao nhóm Toán – Tin học – Khoa học tự nhiên của AI lại dày hơn so với CNTT?', 'top_k': 5}


2025-12-02 11:59:54.609 | INFO     | utils.api_client:ask_question:52 - Response data: {'answer': 'Tôi không có đủ thông tin để trả lời câu hỏi này.', 'sources': [{'section_id': '70676e25b82e', 'title': 'Các môn học tự chọn liên ngành', 'hierarchy_path': 'Ngành Trí Tuệ Nhân Tạo > Tỷ lệ các khối kiến thức > Khối kiến thức giáo dục chuyên nghiệp > Các môn học tự chọn liên ngành', 'text_preview': '[Ngành Trí Tuệ Nhân Tạo > Tỷ lệ các khối kiến thức > Khối kiến thức giáo dục chuyên nghiệp > Các môn học tự chọn liên ngành]\n\n\\- Tổng số tín chỉ cho các môn tự chọn liên ngành đạt tối thiểu\xa0**8**\xa0tín chỉ.\n\\- Sinh viên có thể chọn học:\no Các môn học liên ngành trong danh sách khuyến nghị được liệt kê trong bảng sau.\no Hoặc, các môn học cơ sở ngành và chuyên ngành trong các chương trình đào tạo đại học ngành khác của trường.\no Hoặc, các môn học trong các chương trình đào tạo thạc sĩ của trường.\no Hoặc, các môn học tự chọn ngành Trí tuệ nhân tạo, nếu các m

  ✅ Got answer (49 chars)
[1] Generating answer for: So sánh tổng số tín chỉ đại cương của CNTT và TTNT...
{'query': 'So sánh tổng số tín chỉ đại cương của CNTT và TTNT: ngành nào nặng hơn và chênh lệch bao nhiêu?', 'top_k': 5}


2025-12-02 11:59:55.941 | INFO     | utils.api_client:ask_question:52 - Response data: {'answer': 'Tôi không có đủ thông tin để trả lời câu hỏi này.', 'sources': [{'section_id': '25cae7ce09b3', 'title': 'Tỷ lệ các khối kiến thức', 'hierarchy_path': 'Ngành Kỹ Thuật Máy Tính > Tỷ lệ các khối kiến thức', 'text_preview': '[Ngành Kỹ Thuật Máy Tính > Tỷ lệ các khối kiến thức]\n\nChương trình cho phép sinh viên đăng ký các môn học tương đương hoặc thay thế nhằm tích lũy tín chỉ đầy đủ và linh hoạt. Các môn học tương đương có thể bao gồm tự chọn chuyên ngành, tự chọn tự do, Đồ án, Thực tập và Khóa luận tốt nghiệp. Việc xét tương đương dựa trên nội dung, số tín chỉ và định hướng chuyên môn của môn học gốc, giúp sinh viên hoàn thiện chương trình đào tạo một cách hệ thống và chuyên sâu.\n\n[Bảng 1] Bảng 10 hàng x 4 cột. Các cột: Khối kiến thức, , Khối lượng, ...\n: Khối kiến thức, : Khối lượng\nKhối lượng: Tổng số tín chỉ, : %\nKhối kiến thức: Khối kiến thức giáo

...

  ✅ Got answer (133 chars)
[10] Generating answer for: Môn xử lý tiếng nói có bao nhiêu tín chỉ nên học v...
{'query': 'Môn xử lý tiếng nói có bao nhiêu tín chỉ nên học vào học kì nào?', 'top_k': 5}


2025-12-02 12:00:07.518 | INFO     | utils.api_client:ask_question:52 - Response data: {'answer': 'Tôi không có đủ thông tin để trả lời câu hỏi này.', 'sources': [{'section_id': '72a68f6d81fd', 'title': 'Học kì 6', 'hierarchy_path': 'Ngành Công Nghệ Thông Tin > Kế hoạch giảng dạy mẫu > Học kì 6', 'text_preview': '[Ngành Công Nghệ Thông Tin > Kế hoạch giảng dạy mẫu > Học kì 6]\n\n\n\n[Bảng 1] Bảng 6 hàng x 5 cột. Các cột: Mã môn, Tên môn học, TC, ...\n: Mã môn, : Tên môn học, : TC, : LT, : TH\nMã môn: SS010, Tên môn học: Lịch sử Đảng Cộng sản Việt Nam, TC: 2, LT: 2, TH: 0\nMã môn: IE105, Tên môn học: Nhập môn bảo đảm và an ninh thông tin, TC: 4, LT: 3, TH: 1\nMã môn: IE108, Tên môn học: Phân tích thiết kế phần mềm, TC: 4, LT: 3, TH: 1\nMã môn: PE232, Tên môn học: Giáo dục thể chất 2, TC: Tính riêng\nTên môn học: Các môn học chuyên ngành (**), TC: ≥\xa010\nTên môn học: Tổng số tín chỉ HK6, TC: ≥ 16', 'score': 0.80504847}, {'section_id': '6d416d0a683c', 'title': 'Các khối kiến thức', 'hie

  ✅ Got answer (49 chars)
[11] Generating answer for: Sinh viên CNTT muốn theo hướng Khoa học thông tin ...
{'query': 'Sinh viên CNTT muốn theo hướng Khoa học thông tin phải hoàn thành tối thiểu bao nhiêu tín chỉ nhóm đó?', 'top_k': 5}


2025-12-02 12:00:08.902 | INFO     | utils.api_client:ask_question:52 - Response data: {'answer': 'Sinh viên ngành Công Nghệ Thông Tin muốn theo hướng Khoa học thông tin phải hoàn thành tối thiểu 6 tín chỉ từ nhóm môn học chuyên ngành được quy định.', 'sources': [{'section_id': 'da173e525700', 'title': 'Nhóm các môn học chuyên ngành', 'hierarchy_path': 'Ngành Công Nghệ Thông Tin > Các khối kiến thức > Khối kiến thức giáo dục chuyên nghiệp > Nhóm các môn học chuyên ngành', 'text_preview': '[Ngành Công Nghệ Thông Tin > Các khối kiến thức > Khối kiến thức giáo dục chuyên nghiệp > Nhóm các môn học chuyên ngành]\n\nChia thành 2 hướng — sinh viên có thể học 1 hướng hoặc kết hợp cả 2:\n\n- Hướng Truyền thông xã hội & Công nghệ Web:\n  Web, di động, mạng xã hội, SEO, hệ khuyến nghị, Java, CRM, tư duy thiết kế,…\n\n- Hướng Khoa học thông tin:\n  Python, dữ liệu lớn, phân tích và trực quan dữ liệu, GIS 3D, khai phá dữ liệu, kiến trúc hệ thống,…\n\nSinh viên phải lấy 6–10 tín ch

  ✅ Got answer (150 chars)

✅ Generated 12 answers
✅ Saved 12 responses to ../data/testcase/multi_hop/results_edited_prompt.json


In [ ]:
import utils.convert_results_to_csv as convert_utils

In [ ]:
convert_utils.convert_json_to_csv(
    json_file="../data/rag_results.json",
    csv_file="../data/open_rag_eval_results/rag_results.csv"
)

✅ Converted 6 queries to ../data/open_rag_eval_results/rag_results.csv
   Total rows: 30


In [ ]:
import json
import csv
file_path = "../data/testcase/factual_testcase/questions.json"

# Đọc file JSON
with open(file_path, "r", encoding="utf-8") as f:
    questions = json.load(f)

with open("questions.csv", "w", newline="", encoding="utf-8-sig") as csvfile:
    writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)  # Bao quanh tất cả giá trị bằng ""
    writer.writerow(["query"])

    for row in questions:
        # Lấy nội dung câu hỏi và loại bỏ newline hoặc dấu đặc biệt gây lỗi
        question = row.get("question", "").replace("\n", " ").replace("\r", " ").strip()
        writer.writerow([question])

## RAPTOR

In [ ]:
rag_responses = []
# Generate answers for first N test cases
for i, tc in enumerate(questions):
    question_id = tc['question_id']
    question = tc['question']
    print(f"[{question_id}] Generating answer for: {question[:50]}...")
    
    try:
        # Call RAG API
        response = api_client.ask_question(question)
        print(response)
        
        # Store result
        rag_responses.append({
            'question_id':question_id,
            'question': question,
            'answer': response['answer'],
            'sources': response['sources'],
            'contexts': [context.replace("\n", " ") for context in response.get('contexts', [])],
        })
        
        print(f"  ✅ Got answer ({len(response['answer'])} chars)")
        
    except Exception as e:
        print(f"  ❌ Error: {e}")
        rag_responses.append({
            'question_id': tc['id'],
            'question': question,
            'answer': '',
            'contexts': [],
            'sources': [],
            'error': str(e)
        })

print(f"\n✅ Generated {len([r for r in rag_responses if r.get('answer')])} answers")

2025-12-01 17:07:33.514 | INFO     | utils.api_client:ask_question:43 - Asking question: Ngành Trí tuệ nhân tạo yêu cầu tối thiểu bao nhiêu...
2025-12-01 17:07:33.515 | ERROR    | utils.api_client:ask_question:57 - API request failed: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/query (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10a6d6f10>: Failed to establish a new connection: [Errno 61] Connection refused'))
2025-12-01 17:07:33.516 | INFO     | utils.api_client:ask_question:43 - Asking question: Thời gian đào tạo ngành Công Nghệ Thông Tin kéo dà...
2025-12-01 17:07:33.517 | ERROR    | utils.api_client:ask_question:57 - API request failed: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/query (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10a6d67a0>: Failed to establish a new connection: [Errno 61] Connection refused'))
2025-12-01 17:07:33.517 | 

[F001] Generating answer for: Ngành Trí tuệ nhân tạo yêu cầu tối thiểu bao nhiêu...
{'query': 'Ngành Trí tuệ nhân tạo yêu cầu tối thiểu bao nhiêu tín chỉ để tốt nghiệp?'}
{'success': False, 'error': "HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/query (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10a6d6f10>: Failed to establish a new connection: [Errno 61] Connection refused'))", 'answer': '', 'sources': []}
  ✅ Got answer (0 chars)
[F002] Generating answer for: Thời gian đào tạo ngành Công Nghệ Thông Tin kéo dà...
{'query': 'Thời gian đào tạo ngành Công Nghệ Thông Tin kéo dài bao nhiêu năm?'}
{'success': False, 'error': "HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/query (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10a6d67a0>: Failed to establish a new connection: [Errno 61] Connection refused'))", 'answer': '', 'sources': []}
  ✅ Got answer (

In [ ]:
output_path = "../data/rag_results_raptor.json"

# Ghi ra JSON với định dạng đẹp
with open(output_path, "w", encoding="utf-8-sig") as f:
    json.dump(rag_responses, f, ensure_ascii=False, indent=2)

print(f"✅ Saved {len(rag_responses)} responses to {output_path}")

✅ Saved 18 responses to ../data/rag_results_raptor.json


In [ ]:
import utils.convert_results_to_csv as convert_utils

In [ ]:
convert_utils.convert_json_to_csv_with_contexts(
    json_file="../data/rag_results_raptor.json",
    csv_file="../data/open_rag_eval_results_raptor/rag_results_raptor.csv"
)

✅ Converted 18 queries to ../data/open_rag_eval_results_raptor/rag_results_raptor.csv
   Total rows: 18


In [ ]:
import pandas as pd

file_path = "../data/questions.csv"

# Đọc CSV
df = pd.read_csv(file_path)

# Hiển thị toàn bộ bảng
print(df)



                                               query
0  Ngành Kỹ thuật Máy tính có thể làm những công ...
1  Sinh viên mới vào học nên theo thứ tự học nhữn...
2  Nếu em muốn theo hướng Hệ thống nhúng/IoT thì ...
3  Ngành Kỹ thuật Máy tính có những định hướng ch...
4  Trong Ngành Công Nghệ Thông Tin, môn Thực tập ...
5  Chương trình Trí Tuệ Nhân Tạo có những phương ...
